In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [3]:
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [16]:
from sqlalchemy import create_engine
import logging
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text

# Data Preprocessing

Running this part will download all the files in the local repo, but it will take a long time. 
Depending on the machine it runs on, it might throw error due to large size.

### Clean Weather Data

All weather csv are cleaned and seperated into hours part and days part

In [ ]:
#create the clean version of weather csv and seprated into two parts
def create_weather_csv(filename):
    # make sure the output title is consistent 
    title1 = ""
    title2 = ""
    m = re.search(r'(\d+_weather)', filename)
    if m:
        title1 = m.group(1) + '_hours.csv'
        title2 = m.group(1) + '_days.csv'
    
    
    #this function cleans the weather csv and output 2 csv, one is for daily and one is for hour
    df = pd.read_csv(filename,usecols=['DATE','HourlyWindSpeed','HourlyPrecipitation'])
    #deal with missing value and special character
    df = df.fillna(0)
    df = df.replace('T',0)
    df = df.replace('s','',regex = True)
    #change the data type
    df.DATE = pd.to_datetime(df.DATE)
    df = df.astype({'HourlyPrecipitation':float,'HourlyWindSpeed':float})
    df.set_index('DATE',drop = True).to_csv(title1)
    df = df.resample('D', on='DATE').mean() 
    df = df.rename(mapper = {"HourlyPrecipitation":"DailyPrecipitation", "HourlyWindSpeed":"DailyWindSpeed"},axis = 1)
    df.to_csv(title2)
    
    

In [ ]:
for i in range(2009,2016):
    #create all weather data from 2009 to 2015
    filename = str(i)+"_weather.csv"
    create_weather_csv(filename)

### Clean Uber Data

Clean uber sample data and makes the name consistent to the yellow taxi data

In [ ]:
uber = pd.read_csv('uber_rides_sample.csv')

In [ ]:
#this function gets the distance between two coordinates
def get_distance(lon1,lat1,lon2,lat2):
    from math import sin, cos, sqrt, atan2, radians
    R = 6373
    lon1 = radians(lon1)
    lat1 = radians(lat1)
    lon2 = radians(lon2)
    lat2 = radians(lat2)
    
    dlon = lon1 - lon2
    dlat = lat1 - lat2
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

In [ ]:
#add distance to the dataframe with coordinates
def add_distance(df):
    distance = []
    lon1 = list(df['pickup_longitude'])
    lon2 = list(df['dropoff_longitude'])
    lat1 = list(df['pickup_latitude'])
    lat2 = list(df['dropoff_latitude'])
    for i in range(len(lon1)):
        distance.append(get_distance(lon1[i],lat1[i],lon2[i],lat2[i]))
    df['distance']  = distance

In [ ]:
#create the clean version of uber dataframe
def create_uber(df):
    #clean uber data
    df = df.rename(columns = lambda x: x.strip())
   
    #drop and rename column
    to_drop = [
        "Unnamed: 0",
        "key",
        "passenger_count"
    ]
    
    mapper = {
        "pickup_datetime" :"pickup_time",
        "fare_amount" : "charge"
    }
    df = df.drop(to_drop, axis = 1,errors = "ignore")
    df = df.rename(mapper, axis = 1)
    df.dropna(inplace = True)
    
    #modify datatype
    df = df.astype({"pickup_time":np.datetime64})
    
    
    #add distance
    add_distance(df)
    df.to_csv('uber.csv',index = False)

In [ ]:
create_uber(uber)

### Clean Yellow Taxi Data

find all csv files and filter them, then download as a dataframe and clean the data

In [ ]:
def get_csv_links():
    #this function visits： https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
    #It requests the source code on the website and get all the hrefs related to csv
    #the urls are saved in link_lists
    link_lists = []
    url = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    aclasses = soup.find_all('a')
    for a in aclasses:
        link_lists.append(a.get('href'))
    #Then we filter on link_lists using re because we only want to grab csv for yellow taxi ranging from 2009 - 2015.
    csv_links = []
    pattern = re.compile(r'.yellow_tripdata_(200[9]|201[0-5])-\d\d\.csv$')
    for i in link_lists:
        if re.search(pattern,i):
            csv_links.append(i)
    return csv_links
    

In [ ]:
westlimit=-74.242330; southlimit=40.560445; eastlimit=-73.717047; northlimit=40.908524
#Remove the data that is not within the limits specified above

def fix_longitude(input_longitude):
    try:
        input_longitude = float(input_longitude)
    except:
        return np.NaN
    if input_longitude < westlimit or input_longitude > eastlimit:
        return np.NaN
    return input_longitude


def fix_latitude(input_latitude):
    try:
        input_latitude = float(input_latitude)
    except:
        return np.NaN
    if input_latitude < southlimit or input_latitude > northlimit:
        return np.NaN
    return input_latitude


def fix_df(df):
    df['pickup_longitude']=df['pickup_longitude'].apply(fix_longitude)
    df['dropoff_longitude']=df['dropoff_longitude'].apply(fix_longitude)
    df['pickup_latitude']=df['pickup_latitude'].apply(fix_latitude)
    df['dropoff_latitude']=df['dropoff_latitude'].apply(fix_latitude)
    df.dropna(inplace = True)

In [ ]:
def create_csv(csv_link):
   
    # make sure the output title is consistent 
    title = ""
    m = re.search(r'(yellow.+)', csv_link)
    if m:
        title = m.group(1)
   
    #read the data into a dataframe
    df = pd.read_csv(csv_link,on_bad_lines='skip')
    df = df.rename(columns = lambda x: x.strip())
   
    #drop and rename column
    to_drop = [
        "Unnamed: 0",
        "vendor_name",
        "vendor_id",
        "Vendor_id",
        'VendorID',
        "Trip_distance",
        "Trip_Distance",
        "trip_distance",
        "Rate_Code",
        "store_and_forward",
        "store_and_fwd_flag",
        "Payment_Type",
        "Fare_Amt",
        "surcharge",
        "mta_tax",
        "Tolls_Amt",
        "rate_code",
        "RatecodeID",
        "RateCodeID",
        "payment_type",
        "fare_amount",
        "extra",
        "tolls_amount",
        "improvement_surcharge",
        "Passenger_Count",
        "passenger_count"
    ]
    
    mapper = {
        "Trip_Pickup_DateTime" : "pickup_time",
        "tpep_pickup_datetime" : "pickup_time",
        "pickup_datetime": "pickup_time",
        "dropoff_datetime" : "dropoff_time",
        "Trip_Dropoff_DateTime" : "dropoff_time",
        "tpep_dropoff_datetime" : "dropoff_time",
        "Start_Lon" : "pickup_longitude",
        "Start_Lat" : "pickup_latitude",
        "End_Lon" : "dropoff_longitude",
        "End_Lat" : "dropoff_latitude",
        "Tip_Amt" : 'tip',
        "tip_amount" : "tip",
        "Total_Amt" : "charge",
        "total_amount" : "charge"
    }
    df = df.drop(to_drop, axis = 1,errors = "ignore")
    df = df.rename(mapper, axis = 1)
    df.dropna(inplace = True)
    
    #modify datatype
    df = df.astype({"pickup_time":np.datetime64,"dropoff_time": np.datetime64})
    
    
    #make sure the trip is within(40.560445, -74.242330) and (40.908524, -73.717047)
    fix_df(df)
    
    #sample 3000 rows
    df = df.sample(n=3000)
    
    #add distance
    add_distance(df)
    df.reset_index(inplace = True, drop = True)
    
    df.to_csv(title,index= False)

run this to download all the yellow taxi sample data

In [ ]:
from tqdm import tqdm
links = get_csv_links()
for i in tqdm(range(len(links))):
    create_csv(links[i])


# Storing Data

We first read all csv files into 4 dataframe

In [5]:
def createuber():
    #this function reads in uber.csv and returns the data in pd.DataFrame format
    uber = pd.read_csv('uber.csv')
    uber.pickup_time = pd.to_datetime(uber.pickup_time)
    uber = uber[uber.distance != 0]
    return uber

In [6]:
def createtaxi():
    #this function reads in yellow_trip_{year}-{month}.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        for j in range(1,13):
            if j <10:
                filename = f'yellow_tripdata_{i}-0{j}.csv'
                filenames.append(filename)
            else:
                filename = f'yellow_tripdata_{i}-{j}.csv'
                filenames.append(filename)
    taxi = pd.read_csv('yellow_tripdata_2009-01.csv',usecols = ['pickup_time','dropoff_time','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','charge','tip','distance'],nrows = 2314)
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i],usecols = ['pickup_time','dropoff_time','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','charge','tip','distance'], nrows = 2314)
        taxi = pd.concat([taxi,df])
    taxi.reset_index(inplace = True, drop = True)
    taxi.pickup_time = pd.to_datetime(taxi.pickup_time)
    taxi.dropoff_time = pd.to_datetime(taxi.dropoff_time)
    return taxi

In [7]:
def createhourlyweather():
    #this function reads in {year}_weather_hours.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        filenames.append(f'{i}_weather_hours.csv')
    weather_hourly = pd.read_csv('2009_weather_hours.csv')
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i])
        weather_hourly = pd.concat([weather_hourly,df])
    weather_hourly.reset_index(inplace = True, drop = True)
    weather_hourly.DATE = pd.to_datetime(weather_hourly.DATE)
    weather_hourly['DATE'] = weather_hourly['DATE'].dt.floor('H')
    weather_hourly = weather_hourly.drop_duplicates('DATE',keep = 'first')
    mapper = {'DATE': 'date', "HourlyPrecipitation":'precipitation','HourlyWindSpeed':'windspeed'}
    weather_hourly = weather_hourly.rename(mapper,axis = 1)
    return weather_hourly
    

In [8]:
def createdailyweather():
    #this function reads in {year}_weather_days.csv and returns the data in pd.DataFrame format
    filenames = []
    for i in range(2009,2016):
        filenames.append(f'{i}_weather_days.csv')
    weather_daily = pd.read_csv('2009_weather_days.csv')
    for i in range(1,len(filenames)):
        df = pd.read_csv(filenames[i])
        weather_daily = pd.concat([weather_daily,df])
    weather_daily.reset_index(inplace = True, drop = True)
    weather_daily.DATE = pd.to_datetime(weather_daily.DATE)
    mapper = {'DATE': 'date', "DailyPrecipitation":'precipitation','DailyWindSpeed':'windspeed'}
    weather_daily = weather_daily.rename(mapper,axis = 1)
    return weather_daily

In [9]:
uber = createuber()
taxi = createtaxi()
weather_hourly= createhourlyweather()
weather_daily = createdailyweather()


Now we try to build database and four tables

In [10]:
#build database
engine = create_engine(f"sqlite:///ubertaxi.db", echo=True)
sqllogger = logging.getLogger("sqlalchemy.engine.Engine")
formatter = logging.Formatter("[sqlalchemy] %(message)s")
sqllogger.handlers[0].setFormatter(formatter)
Base = declarative_base()
Base.metadata.create_all(engine, checkfirst=True)
# sessionmaker returns a Session class
Session = sessionmaker(bind=engine)
# and we create an instance of Session
session = Session()

[sqlalchemy] BEGIN (implicit)
[sqlalchemy] COMMIT


In [11]:
def readsqlfile(filename):
    #I am gonna be honest, I found this on stack overflow. Basically it allows the sqlalchemy to execute multiple queries 
    #in one file. 
    sql_file = open('schema.sql','r')

    # Create an empty command string
    sql_command = ''
    for line in sql_file:
        # Ignore commented lines
        if not line.startswith('--') and line.strip('\n'):
            # Append line to the command string
            sql_command += line.strip('\n')
            # If the command string ends with ';', it is a full statement
            if sql_command.endswith(';'):
                # Try to execute statement and commit it
                try:
                    session.execute(text(sql_command))
                    session.commit()
                # Assert in case of error
                except:
                    print('Ops')
                # Finally, clear command string
                finally:
                    sql_command = ''

Read schema file

In [11]:
readsqlfile('schema.sql')

Ops
Ops
Ops
Ops


In [12]:
#add four dataframes into sql tables
#DONT EXECUTE TWINCE!!!!!
'''
uber.to_sql('Uber', engine, if_exists='append',index = False)
taxi.to_sql('yellow_taxi', engine, if_exists='append',index = False)
weather_hourly.to_sql('weather_hourly', engine, if_exists='append',index = False)
weather_daily.to_sql('weather_daily', engine, if_exists='append',index = False)

[sqlalchemy] PRAGMA main.table_info("Uber")
[sqlalchemy] [raw sql] ()
[sqlalchemy] PRAGMA temp.table_info("Uber")
[sqlalchemy] [raw sql] ()
[sqlalchemy] BEGIN (implicit)
[sqlalchemy] 
CREATE TABLE "Uber" (
	charge FLOAT, 
	pickup_time DATETIME, 
	pickup_longitude FLOAT, 
	pickup_latitude FLOAT, 
	dropoff_longitude FLOAT, 
	dropoff_latitude FLOAT, 
	distance FLOAT
)


[sqlalchemy] [no key 0.00068s] ()
[sqlalchemy] COMMIT
[sqlalchemy] BEGIN (implicit)
[sqlalchemy] INSERT INTO "Uber" (charge, pickup_time, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, distance) VALUES (?, ?, ?, ?, ?, ?, ?)
[sqlalchemy] [generated in 1.76549s] ((7.5, '2015-05-07 19:52:06.000000', -73.99981689453125, 40.73835372924805, -73.99951171875, 40.72321701049805, 1.6838511852242786), (7.7, '2009-07-17 20:04:56.000000', -73.994355, 40.728225, -73.99471, 40.750325, 2.458361376443877), (12.9, '2009-08-24 21:45:00.000000', -74.005043, 40.74077, -73.962565, 40.772647, 5.0379582221658445), (5.3, '

# Understanding Data

Simply for test

In [47]:
engine.execute("SELECT COUNT(*) FROM yellow_taxi").fetchall()

[sqlalchemy] SELECT COUNT(*) FROM yellow_taxi
[sqlalchemy] [raw sql] ()


[(194376,)]

In [25]:
#specifically for the file that only contains one command
def readsql(filename):
    with engine.connect() as con:
        with open(filename) as file:
            query = text(file.read())
            return engine.execute(query).fetchall()

### Question 1

In [38]:
readsql('most_popular_hour.sql')

[sqlalchemy] SELECT 
strftime ('%H',pickup_time) as most_popular_hour,
Count(*) as times
FROM
yellow_taxi
WHERE
pickup_time between '2009-01-01' AND '2015-06-30'
GROUP BY strftime ('%H',pickup_time) 
ORDER BY times DESC
LIMIT 1;
[sqlalchemy] [generated in 0.00084s] ()


[('19', 11260)]

### Question 2

In [50]:
readsql('most_popular_day.sql')
#So it is Friday! 

[sqlalchemy] SELECT 
strftime ('%w',pickup_time) as most_popular_day,
Count(*) as times
FROM
Uber
WHERE
pickup_time between '2009-01-01' AND '2015-06-30'
GROUP BY strftime ('%w',pickup_time) 
ORDER BY times DESC
LIMIT 1;
[sqlalchemy] [generated in 0.00136s] ()


[('5', 29996)]